In [1]:
from importlib import reload
import gnn_tracking_hpo
reload(gnn_tracking_hpo)

from gnn_tracking_hpo.util.paths import add_scripts_path, get_config, find_checkpoints

add_scripts_path()

In [2]:
from tune_fixed_ec_sectorized import PretrainedECTrainable

In [5]:
project = "tcn-fixed-ec-ce8de2c0-pruned-broken-dm-metric/"
hash = "14fd4234"

In [6]:
config = get_config(project, hash)

In [7]:
trainable = PretrainedECTrainable(config)

DEBUG: Got config
{'batch_size': 1,
 'ec_pt_thld': 0.9,
 'focal_alpha': 0.4113666449910418,
 'focal_gamma': 1,
 'gnn_tracking_experiments_hash': 'fa26f12ec8c2ca29540c0f4ab8f4061b563eceb2',
 'gnn_tracking_hash': 'c322cac8b366756afd84432c0cd7e64ef49ff239',
 'lr': 0.0005929964890981143,
 'lw_edge': 1.0,
 'm_L_ec': 3,
 'm_alpha_ec': 0.8626651724747356,
 'm_e_dim': 4,
 'm_h_dim': 5,
 'm_hidden_dim': 208,
 'm_interaction_edge_hidden_dim': 82,
 'm_interaction_node_hidden_dim': 35,
 'n_graphs_test': 1,
 'n_graphs_train': 1,
 'n_graphs_val': 1,
 'optimizer': 'adam',
 'scheduler': None,
 'sector': 9,
 'test': False,
 'training_pt_thld': 0.0,
 'training_without_noise': False,
 'training_without_non_reconstructable': False}
INFO: Loading data to cpu memory
INFO: Loading 3 graphs (out of 371 available).
DEBUG: Parameters for data loaders: {'batch_size': 1, 'num_workers': 12}
INFO: Using device cpu
DEBUG: Got config
{'attr_pt_thld': 0.725762815354442,
 'batch_size': 1,
 'ec_hash': 'ce8de2c0',
 'ec_p

In [9]:
trainable.load_checkpoint(find_checkpoints(project, hash)[-1], device="cpu")

In [0]:
from gnn_tracking.training.tcn_trainer import TCNTrainer

In [12]:
trainer = trainable.trainer

In [11]:

TCNTrainer.test_step(trainer)

INFO: Starting hyperparameter scan for clustering
INFO: Completed 12 trials, pruned 0 trials
DEBUG: Evaluating all metrics for best clustering
INFO: Evaluating all metrics took 2.51 seconds
INFO: Clustering hyperparameter scan & metric evaluation took 18.21 seconds


{'acc_pt0.9': 0.9915766114725578,
 'TPR_pt0.9': 0.6166513042491828,
 'TNR_pt0.9': 0.9993733026615738,
 'FPR_pt0.9': 0.0006266973384261488,
 'FNR_pt0.9': 0.38334869575081715,
 'balanced_acc_pt0.9': 0.8080123034553784,
 'F1_pt0.9': 0.7462812059370595,
 'MCC_pt0.9': 0.7617708413077519,
 'max_ba_pt0.9': 0.9634767548027661,
 'max_ba_loc_pt0.9': 0.060738473563738495,
 'max_f1_pt0.9': 0.790313509590724,
 'max_f1_loc_pt0.9': 0.2567183664743451,
 'max_mcc_pt0.9': 0.7907844708869616,
 'max_mcc_loc_pt0.9': 0.2766732139431912,
 'tpr_eq_tnr_pt0.9': 0.9594584658405834,
 'tpr_eq_tnr_loc_pt0.9': 0.07428446471475172,
 'roc_auc_pt0.9': 0.9935594130896382,
 'roc_auc_5FPR_pt0.9': 0.9452476673066776,
 'roc_auc_10FPR_pt0.9': 0.9683578182313857,
 'roc_auc_20FPR_pt0.9': 0.9825580196863978,
 'acc_pt1.5': 0.9927851774995541,
 'TPR_pt1.5': 0.6444523978964606,
 'TNR_pt1.5': 0.9992250483767491,
 'FPR_pt1.5': 0.0007749516232510302,
 'FNR_pt1.5': 0.3555476021035394,
 'balanced_acc_pt1.5': 0.8218387231366049,
 'F1_pt

In [13]:
import torch

trainer.model.eval()

sel = 3
with torch.no_grad():
    loader = trainer.val_loader
    for idx, data in enumerate(loader):
        if idx < sel:
            continue
        model_output = trainer.evaluate_model(data, mask_pids_reco=False)
        if idx == sel:
            break

In [14]:
mo = trainable.trainer.evaluate_model(data)

In [17]:
mo["x"].shape

torch.Size([149, 5])

In [18]:
import matplotlib.pyplot as plt

In [34]:
mo["x"].shape

torch.Size([149, 5])

In [32]:
len(mo["particle_id"])

2167

In [40]:
list(mo.keys())

['truth_cuts_applied',
 'w',
 'x',
 'beta',
 'pred',
 'ec_hit_mask',
 'ec_edge_mask',
 'y',
 'particle_id',
 'track_params',
 'pt',
 'reconstructable',
 'edge_index',
 'sector',
 'node_features']

In [63]:
(mo["pt"] > 0.9).sum()

tensor(212)

In [60]:
mask = mo["particle_id"][mo["ec_hit_mask"]] == mo["particle_id"][mo["ec_hit_mask"]][10]

In [61]:
mask.sum()

tensor(5)

In [62]:
%matplotlib notebook
with torch.no_grad():
    plt.plot(mo["x"][~mask, 0], mo["x"][~mask,1 ], ls="", marker="o", alpha=0.5, color="black")
    plt.plot(mo["x"][mask, 0], mo["x"][mask,1 ], ls="", marker="o", alpha=0.5, color="red")

<IPython.core.display.Javascript object>

In [65]:
from gnn_tracking.postprocessing.dbscanscanner import dbscan

labels = dbscan(mo["x"].detach().numpy(), eps=0.9)

In [66]:
from gnn_tracking.metrics.cluster_metrics import tracking_metrics

m = mo["ec_hit_mask"]
tracking_metrics(
    truth=mo["y"][m],
    predicted=labels,
    pts=mo["pt"][m],
    reconstructable=mo["reconstructable"][m],
    pt_thlds=[0.9]
)

IndexError: The shape of the mask [2167] at index 0 does not match the shape of the indexed tensor [117682] at index 0